In [2]:
# import files
import pandas as pd
import re
import os
import json

In [7]:
# importing file into pandas
file_name = 'data_column_field_paths.csv'

csv_file_path = f"input\\{file_name}"

# converting data to pandas dataframe 
df = pd.read_csv(csv_file_path)
df

C:\Users\IzzatSofian\AppData\Local\Temp\ipykernel_27364\666753157.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
1,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,record,record,STRING,NaN,NaN,NaN
2,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,reasons,reasons,STRING,NaN,NaN,NaN
3,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,_sdc_rejected_at,_sdc_rejected_at,TIMESTAMP,NaN,NaN,NaN
4,x-marketing,brp_abx_linkedin_ads,campaigns,last_modified_time,last_modified_time,TIMESTAMP,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
606873,x-marketing,everdriven_mysql,content_wise_mapping,_intentkeyword,_intentkeyword,STRING,NaN,NaN,NaN
606874,x-marketing,everdriven_mysql,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
606875,x-marketing,everdriven_mysql,_sdc_rejected,record,record,STRING,NaN,NaN,NaN
606876,x-marketing,everdriven_mysql,_sdc_rejected,reasons,reasons,STRING,NaN,NaN,NaN


In [10]:
# extract ARRAY/STRUCT string as data type
def extract_prefix(value):
    if value.startswith('STRUCT'):
        return 'STRUCT'
    elif value.startswith('ARRAY'):
        return 'ARRAY'
    else:
        return value

# apply function to dataframe
df['data_type'] = df['data_type'].apply(extract_prefix)
df

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
1,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,record,record,STRING,NaN,NaN,NaN
2,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,reasons,reasons,STRING,NaN,NaN,NaN
3,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,_sdc_rejected_at,_sdc_rejected_at,TIMESTAMP,NaN,NaN,NaN
4,x-marketing,brp_abx_linkedin_ads,campaigns,last_modified_time,last_modified_time,TIMESTAMP,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
606873,x-marketing,everdriven_mysql,content_wise_mapping,_intentkeyword,_intentkeyword,STRING,NaN,NaN,NaN
606874,x-marketing,everdriven_mysql,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
606875,x-marketing,everdriven_mysql,_sdc_rejected,record,record,STRING,NaN,NaN,NaN
606876,x-marketing,everdriven_mysql,_sdc_rejected,reasons,reasons,STRING,NaN,NaN,NaN


In [11]:
# removing bigquery specific metadata column
# we do not need this since these are not relevant
df = df[~df['field_path'].isin(['_sdc_rejected_at',
                           '_sdc_table_version',
                           '_sdc_received_at',
                           '_sdc_sequence',
                           '_sdc_batched_at',
                           '_sdc_extracted_at',
                           '_sdc_deleted_at'])]

# resetting index to its true row number
df = df.reset_index(drop=True)
df

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
1,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,record,record,STRING,NaN,NaN,NaN
2,x-marketing,brp_abx_linkedin_ads,_sdc_rejected,reasons,reasons,STRING,NaN,NaN,NaN
3,x-marketing,brp_abx_linkedin_ads,campaigns,last_modified_time,last_modified_time,TIMESTAMP,NaN,NaN,NaN
4,x-marketing,brp_abx_linkedin_ads,campaigns,offsite_delivery_enabled,offsite_delivery_enabled,BOOL,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
588890,x-marketing,everdriven_mysql,content_wise_mapping,_title,_title,STRING,NaN,NaN,NaN
588891,x-marketing,everdriven_mysql,content_wise_mapping,_intentkeyword,_intentkeyword,STRING,NaN,NaN,NaN
588892,x-marketing,everdriven_mysql,_sdc_rejected,table_name,table_name,STRING,NaN,NaN,NaN
588893,x-marketing,everdriven_mysql,_sdc_rejected,record,record,STRING,NaN,NaN,NaN


In [12]:
schema_name = 'x3x_hubspot'
table_name = 'contacts'

# your final table, please specify the dataset/crm, table name, or column(optional)
df = df[
    # specify dataset/crm
    (df['table_schema'] == schema_name) &
    # specify table name
    (df['table_name'] == table_name) 
    # specify column name (optional)
    # (df['column_name'].str.contains('properties'))
]
df

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
404703,x-marketing,x3x_hubspot,contacts,vid,vid,INT64,NaN,NaN,NaN
404704,x-marketing,x3x_hubspot,contacts,versiontimestamp,versiontimestamp,TIMESTAMP,NaN,NaN,NaN
404705,x-marketing,x3x_hubspot,contacts,properties,properties,STRUCT,NaN,NaN,NaN
404706,x-marketing,x3x_hubspot,contacts,properties,properties.currentlyinworkflow,STRUCT,NaN,NaN,NaN
404707,x-marketing,x3x_hubspot,contacts,properties,properties.currentlyinworkflow.value,STRING,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
406286,x-marketing,x3x_hubspot,contacts,property_hs_registered_member,property_hs_registered_member.value,FLOAT64,NaN,NaN,NaN
406287,x-marketing,x3x_hubspot,contacts,property_hs_membership_has_accessed_private_co...,property_hs_membership_has_accessed_private_co...,STRUCT,NaN,NaN,NaN
406288,x-marketing,x3x_hubspot,contacts,property_hs_membership_has_accessed_private_co...,property_hs_membership_has_accessed_private_co...,FLOAT64,NaN,NaN,NaN
406289,x-marketing,x3x_hubspot,contacts,property_hs_quarantined_emails,property_hs_quarantined_emails,STRUCT,NaN,NaN,NaN


In [13]:
#generate the data_output directory if not yet exist
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

path_to_output_file ="output\\"

#generate CSV file by date time
from datetime import datetime
csv_file_name = datetime.now().strftime('%Y%m%d-%H%M%S')+".csv"

# name the csv output name
csv_file_path = path_to_output_file + csv_file_name

# output syntax
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved to: {csv_file_path}")

CSV file saved to: output\20240902-153100.csv
